In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from urllib.parse import unquote
from io import StringIO
import pandas as pd
import re
from datetime import date
import os.path
from os import path
import platform

In [3]:
outputfilename ="./data/"+symbol+"_"+date.today().strftime("%d_%b_%Y")+".csv"

In [4]:
if path.exists(outputfilename):
    nseData = pd.read_csv(outputfilename)
else:
    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.get("https://www.nseindia.com/products/content/equities/equities/eq_security.htm")



    #assert "Python" in driver.title
    elem = driver.find_element_by_name("symbol")
    elem.clear()
    elem.send_keys(symbol)

    dateRange = Select(driver.find_element_by_name("dateRange"))
    dateRange.select_by_value("24month")


    dateRange = Select(driver.find_element_by_name("series"))
    dateRange.select_by_value("EQ")

    #submitBtn = driver.find_element_by_id("submitMe")
    #elem.submit()

    driver.execute_script("submitData()")



    #dateRange.set_attribute()
    #print(dateRange.get_attribute("value"))

    #elem.send_keys(Keys.RETURN)
    #assert "No results found." not in driver.page_source
    #driver.close()

    xpathStr = "//div[@class='historic-bar']//span[@class='download-data-link']"

    if False:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpathStr))
            )
        finally:
            driver.quit()

    driver.implicitly_wait(10)     
    driver.find_element_by_xpath(xpathStr).click()

    data = driver.find_element_by_xpath(xpathStr+"/a").get_attribute('href')

    dataDecoded = unquote(data)

    dataDecoded = re.sub("data:application/csv;charset=utf-8,","", dataDecoded)
    nseData = pd.read_csv(StringIO(dataDecoded), sep=",")
    nseData.set_index("Date", inplace=True)

    nseData.to_csv(outputfilename)
    driver.quit()